In [1]:
import os
import pandas as pd
import numpy as np
path = "./home/users/khwong352/FYP/ee5434/"
def read_data(partition , data_path=path):
  data = []
  for fn in os.listdir(os.path.join(data_path, partition)):
    with open(os.path.join(data_path, partition, fn)) as f:
      data.append(pd.read_csv(f, index_col=None))
  return pd.concat(data)



In [2]:
# reading all data_partitions
# path = "../input/pfam-seed-random-split/random_split/random_split"
df_test = pd.read_csv('/home/users/khwong352/FYP/ee5434/test.csv')
df_val = pd.read_csv('/home/users/khwong352/FYP/ee5434/val.csv')
df_train = pd.read_csv('/home/users/khwong352/FYP/ee5434/train.csv')
df_sample = pd.read_csv('/home/users/khwong352/FYP/ee5434/sampleSubmission.csv')

#df_train.head(5)
#print(df_train['family_accession'].unique())
#df_train['family_accession'].nunique()
#print(df_train)
#print(df_val)

In [3]:
df_test.head(5)

,ID,reads
0,0,TAATAATTCGCCAATTAAATCATATTTTTAATATGATTTAATTGGC...
1,1,ATTATTTTCCTTGTAGAATTTGGTCTGGACTTTTGCCCTGTGCAAT...
2,2,GGCGCTGTGCTGCTTCTTGGCCGGCGCGGCCGACGCCTACCGGTAC...
3,3,TACGTTTATTTTGGATCCATTCGATCCCCTGAGTCACCATTGGCAG...
4,4,GAAAAGATGCGCGCGACCATTTCGTCGGGCAGCGTCGCGAACAGGT...


In [4]:
len(df_test)

41211

In [5]:
def calc_unique_cls(train, test, val):
  """
  Prints # unique classes in data sets.
  """
  train_unq = np.unique(train['label'].values)
  val_unq = np.unique(val['label'].values)
  test_unq = np.unique(test['ID'].values)

  print('Number of unique classes in Train: ', len(train_unq))
  print('Number of unique classes in Val: ', len(val_unq))
  print('Number of unique classes in Test: ', len(test_unq))

In [6]:
classes = df_train['label'].value_counts()[:1000].index.tolist()
len(classes)

3

In [7]:
# Filtering data based on considered 1000 classes.
train_sm = df_train.loc[df_train['label'].isin(classes)].reset_index()
val_sm = df_val.loc[df_val['label'].isin(classes)].reset_index()
test_sm = df_test.loc[df_test['ID'].isin(classes)].reset_index()

print('Data size after considering 1000 classes for each data split:')
print('Train size :', len(train_sm))
print('Val size :', len(val_sm))
print('Test size :', len(test_sm))

Data size after considering 1000 classes for each data split:
Train size : 1321357
Val size : 170665
Test size : 3


In [8]:
calc_unique_cls(train_sm, test_sm, val_sm)

Number of unique classes in Train:  3
Number of unique classes in Val:  3
Number of unique classes in Test:  3


In [9]:
codes = ['A', 'C', 'G', 'T']

def create_dict(codes):
  char_dict = {}
  for index, val in enumerate(codes):
    char_dict[val] = index+1

  return char_dict

char_dict = create_dict(codes)

print(char_dict)

{'A': 1, 'C': 2, 'G': 3, 'T': 4}


In [10]:
import numpy as np

def integer_encoding(data):
  """
  - Encodes code sequence to integer values.
  - 20 common amino acids are taken into consideration
    and rest 4 are categorized as 0.
  """

  encode_list = []
  for row in data['reads']:
    row_encode = []
    for code in row:
      row_encode.append(char_dict.get(code, 0))
    encode_list.append(np.array(row_encode))

  return encode_list

train_encode = integer_encoding(train_sm) 
val_encode = integer_encoding(val_sm) 
test_encode = integer_encoding(test_sm)

In [11]:
#train_encode[1]

In [12]:
import tensorflow as tf
#from tensorflow.kears.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import sequence as sequence

max_length = 250
train_pad = sequence.pad_sequences(train_encode, maxlen=max_length, padding='post', truncating='post')
val_pad = sequence.pad_sequences(val_encode, maxlen=max_length, padding='post', truncating='post')
test_pad = sequence.pad_sequences(test_encode, maxlen=max_length, padding='post', truncating='post')
#print(train_pad)
print(train_pad.shape, val_pad.shape, test_pad.shape)

(1321357, 250) (170665, 250) (3, 250)


In [13]:
#train_pad[0]

In [14]:

# One hot encoding of sequences
from tensorflow.keras.utils import to_categorical
train_ohe = to_categorical(train_pad)
val_ohe = to_categorical(val_pad)
test_ohe = to_categorical(test_pad)

train_ohe.shape, test_ohe.shape, test_ohe.shape

((1321357, 250, 5), (3, 250, 5), (3, 250, 5))

In [15]:
# label/integer encoding output variable: (y)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y_train_le = le.fit_transform(train_sm['label'])
y_val_le = le.transform(val_sm['label'])
y_test_le = le.transform(test_sm['ID'])

y_train_le.shape, y_val_le.shape, y_test_le.shape

((1321357,), (170665,), (3,))

In [16]:

print('Total classes: ', len(le.classes_))
# le.classes_

Total classes:  3


In [17]:

# One hot encoding of outputs
y_train = to_categorical(y_train_le)
y_val = to_categorical(y_val_le)
y_test = to_categorical(y_test_le)

y_train.shape, y_val.shape, y_test.shape

((1321357, 3), (170665, 3), (3, 3))

In [18]:
# Utility function: plot model's accuracy and loss

# https://realpython.com/python-keras-text-classification/
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  x = range(1, len(acc) + 1)

  plt.figure(figsize=(12, 5))
  plt.subplot(1, 2, 1)
  plt.plot(x, acc, 'b', label='Training acc')
  plt.plot(x, val_acc, 'r', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()

  plt.subplot(1, 2, 2)
  plt.plot(x, loss, 'b', label='Training loss')
  plt.plot(x, val_loss, 'r', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

In [19]:
# Utility function: Display model score(Loss & Accuracy) across all sets.

def display_model_score(model, train, val, test, batch_size):

  train_score = model.evaluate(train[0], train[1], batch_size=batch_size, verbose=1)
  print('Train loss: ', train_score[0])
  print('Train accuracy: ', train_score[1])
  print('-'*70)

  val_score = model.evaluate(val[0], val[1], batch_size=batch_size, verbose=1)
  print('Val loss: ', val_score[0])
  print('Val accuracy: ', val_score[1])
  print('-'*70)
  
  test_score = model.evaluate(test[0], test[1], batch_size=batch_size, verbose=1)
  print('Test loss: ', test_score[0])
  print('Test accuracy: ', test_score[1])

In [20]:
def residual_block(data, filters, d_rate):
  """
  _data: input
  _filters: convolution filters
  _d_rate: dilation ratees
  """

  shortcut = data

  bn1 = BatchNormalization()(data)
  act1 = Activation('relu')(bn1)
  conv1 = Conv1D(filters, 1, dilation_rate=d_rate, padding='same', kernel_regularizer=l2(0.001))(act1)

  #bottleneck convolution
  bn2 = BatchNormalization()(conv1)
  act2 = Activation('relu')(bn2)
  conv2 = Conv1D(filters, 3, padding='same', kernel_regularizer=l2(0.001))(act2)

  #skip connection
  x = Add()([conv2, shortcut])

  return x

In [24]:
# model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Activation
from tensorflow.keras.layers import Conv1D, Add, MaxPooling1D, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

x_input = Input(shape=(250, 5))

#initial conv
conv = Conv1D(128, 1, padding='same')(x_input) 

# per-residue representation
res1 = residual_block(conv, 128, 2)
res2 = residual_block(res1, 128, 3)

x = MaxPooling1D(3)(res2)
x = Dropout(0.5)(x)

# softmax classifier
x = Flatten()(x)
x_output = Dense(3, activation='softmax', kernel_regularizer=l2(0.0001))(x)

model2 = Model(inputs=x_input, outputs=x_output)
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 250, 5)]     0                                            
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 250, 128)     768         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 250, 128)     512         conv1d_5[0][0]                   
__________________________________________________________________________________________________
activation_4 (Activation)       (None, 250, 128)     0           batch_normalization_4[0][0]      
____________________________________________________________________________________________

In [25]:
# Early Stopping
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [26]:
y_pred = model2.predict(test_ohe)

In [27]:

history2 = model2.fit(
    train_ohe, y_train,
    epochs=1, batch_size=256,
    validation_data=(val_ohe, y_val),
    callbacks=[es]
    )

5162/5162 [==============================] - 1758s 340ms/step - loss: 0.6306 - accuracy: 0.8173 - val_loss: 0.6304 - val_accuracy: 0.8580


In [41]:
import tensorflow_addons as tfa
metric = tfa.metrics.F1Score(num_classes=3, average='macro', threshold=0.5)
metric.update_state(y_test, y_pred)
result = metric.result()
result.numpy()

0.33333334

In [42]:
m = tf.keras.metrics.Recall()
m.update_state(y_test, y_pred)
m.result().numpy()

0.33333334

In [44]:
m = tf.keras.metrics.Precision()
m.update_state(y_test, y_pred)
m.result().numpy()

1.0

In [46]:
m = tf.keras.metrics.Accuracy()
m.update_state(y_test, y_pred)
m.result().numpy()

0.0

InvalidArgumentError: Dimensions [0,2) of indices[shape=[3,2,3]] must match dimensions [0,2) of updates[shape=[3,3]] [Op:ScatterNd]

In [ ]:
# saving model weights.
model2.save_weights('/home/users/khwong352/FYP/')


In [ ]:

display_model_score(
    model2,
    [train_ohe, y_train],
    [val_ohe, y_val],
    [test_ohe, y_test],
    256)

In [ ]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ['Sr.no', 'Model', 'Train Acc', 'Val Acc','Test Acc']

x.add_row(['1.', 'Bidirectional LSTM', '0.964', '0.957', '0.958'])
x.add_row(['2.', 'ProtCNN', '0.996', '0.988', '0.988'])

print(x)